# Stress Testing 

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [66]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [67]:
relative_weights = [0.3, 0.3, 0.01, 0.09, 0.3]
absolute_weights = [300000, 300000, 10000, 90000, 300000]

In [79]:
#
change = 1.2

stock_returns = np.log(data[['SPY', 'AEX', 'RDSA.AS', 'HSI']]).diff()
stock_returns[['SPY', 'HSI']] + np.log(data[['USDEUR', 'HKDEUR']]).diff().values
yields = data['YIELDS']*change / 100


In [80]:
# Real losses

losses = np.exp(stock_returns) - 1
until_maturity = pd.Series(np.arange(12, 0, -1/(250))[:len(data)])
until_maturity.index = data.index
losses['bonds'] = np.exp(-until_maturity * yields.diff() + (yields + yields.diff()) / 250) - 1
losses = -losses

In [81]:
total_losses = (losses * relative_weights).sum(axis=1)
total_losses = total_losses * 1000000

In [82]:
#total losses last 5 years
middle_index = len(total_losses)//2
total_losses_5 = total_losses[:middle_index]
total_losses_5

Date
2010-01-01       0.000000
2010-01-04     -54.278006
2010-01-05   -3007.818704
2010-01-06     667.149584
2010-01-07   -1422.366697
                 ...     
2015-02-24   -3247.012000
2015-02-25    -113.859039
2015-02-26   -3595.654775
2015-02-27     778.669922
2015-03-02   -2904.287275
Length: 1496, dtype: float64

In [83]:
total_losses = total_losses.sort_values(ascending=True)
total_losses

Date
2020-03-24   -60334.713581
2011-11-30   -27417.752136
2020-03-13   -24891.630573
2011-11-28   -22952.640055
2020-03-17   -22597.339920
                  ...     
2016-06-24    28702.891293
2020-03-18    34434.503218
2020-03-16    48495.510710
2020-03-09    51857.633150
2020-03-12    65863.260073
Length: 2992, dtype: float64

In [84]:
var_97 = total_losses.quantile(0.025)
var_99 = total_losses.quantile(0.01)

print("10 Years:\n",
"Var 97.5%:\t", var_97, "\n Var 99%:\t", var_99)

10 Years:
 Var 97.5%:	 -10899.95387367169 
 Var 99%:	 -14460.796487448319
